In [293]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../utils/")
import Toolbox as tb
from Toolbox import *

In [294]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Cargar el archivo CSV
data = pd.read_csv("../data/wines_dataset.csv", sep="|")

"""
División de datos en entrenamiento y prueba.

Se toma el 80% de los datos para entrenamiento y el 20% restante para prueba.

Parámetros:
- test_size (float): Proporción del conjunto de prueba (0.2 = 20% de los datos).
- random_state (int): Semilla para la reproducibilidad de la división.

Salida:
- train (DataFrame): Conjunto de entrenamiento.
- test (DataFrame): Conjunto de prueba.
"""
train, test = train_test_split(data, test_size=0.2, random_state=42)

"""
Guardado de los conjuntos de datos en archivos CSV.

Los archivos se almacenan en la carpeta "../data/" con los nombres:
- wines_train.csv → Contiene el 80% de los datos para entrenamiento.
- wines_test.csv → Contiene el 20% de los datos para prueba.

index=False evita que se guarde el índice en los archivos.
"""
train.to_csv(os.path.join("../data/", 'wines_train.csv'), index=False)
test.to_csv(os.path.join("../data/", 'wines_test.csv'), index=False)

In [295]:
# Cargar el conjunto de datos de entrenamiento desde el archivo CSV
df = pd.read_csv("../data/wines_train.csv")
y_train_cat = df["quality"]
y_train_reg = df["alcohol"]
# Mostrar el DataFrame cargado


In [296]:
# Cargar el conjunto de datos de prueba desde el archivo CSV
df_test = pd.read_csv("../data/wines_test.csv")
y_test = df_test["quality"]
# Mostrar el DataFrame cargado
df_test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.4,0.320,0.27,1.4,0.049,38.0,173.0,0.99335,3.03,0.52,9.3,5,white
1,6.6,0.340,0.24,3.3,0.034,29.0,99.0,0.99031,3.10,0.40,12.3,7,white
2,6.4,0.320,0.35,4.8,0.030,34.0,101.0,0.99120,3.36,0.60,12.5,8,white
3,6.8,0.230,0.32,1.6,0.026,43.0,147.0,0.99040,3.29,0.54,12.5,6,white
4,6.7,0.340,0.26,1.9,0.038,58.0,138.0,0.98930,3.00,0.47,12.2,7,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,7.6,0.285,0.32,14.6,0.063,32.0,201.0,0.99800,3.00,0.45,9.2,5,white
1296,11.6,0.470,0.44,1.6,0.147,36.0,51.0,0.99836,3.38,0.86,9.9,4,red
1297,10.2,0.340,0.48,2.1,0.052,5.0,9.0,0.99458,3.20,0.69,12.1,7,red
1298,6.2,0.460,0.17,1.6,0.073,7.0,11.0,0.99425,3.61,0.54,11.4,5,red


In [297]:
# Genera un resumen descriptivo del DataFrame utilizando la función tb.describe_df(df).
# 
#  Muestra información sobre:
#    - DATA_TYPE: Tipo de dato de cada columna (numérico, categórico, etc.).
#    - MISSINGS (%): Porcentaje de valores nulos en cada columna.
#    - UNIQUE_VALUES: Cantidad de valores únicos en cada columna.
#    - CARDIN (%): Cardinalidad relativa (valores únicos / total de filas).
# 
# Útil para comprender la estructura de los datos antes del preprocesamiento.

tb.describe_df(df)

COL_N,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
DATA_TYPE,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,object
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,100,177,87,308,194,132,274,951,106,106,102,7,2
CARDIN (%),0.02,0.03,0.02,0.06,0.04,0.03,0.05,0.18,0.02,0.02,0.02,0.0,0.0


In [298]:
# Definición de variables objetivo para el análisis:

# target_clf = "quality" → Problema de Clasificación
#    - La variable `quality` representa la calidad del vino.
#    - Es una variable categórica con 7 valores únicos (multiclase).
#    - Se utilizarán modelos de clasificación, como RandomForestClassifier o XGBoost.
#    - Es necesario aplicar OneHotEncoder a variables categóricas y StandardScaler a las numéricas.

# target_reg = "alcohol" → Problema de Regresión
#    - La variable `alcohol` indica el porcentaje de alcohol en el vino.
#    - Es una variable numérica continua.
#    - Se utilizarán modelos de regresión, como RandomForestRegressor o LinearRegression.
#    - Requiere escalado de las variables numéricas (StandardScaler) y codificación de las categóricas (OneHotEncoder).

# 📌 Ambos problemas requieren preprocesamiento adecuado según el tipo de modelo utilizado.

target_clf = "quality"
target_reg = "alcohol"

In [299]:
# Definición de las características (features) utilizadas en la clasificación:

# features_cat_clf → Variables categóricas
#    - Contiene solo la variable "class" (tipo de vino: tinto o blanco).
#    - Se debe transformar con OneHotEncoder para convertirla en variables numéricas.

# features_num_clf_1 → Variables numéricas principales
#    - Incluye medidas químicas clave como "volatile acidity", "chlorides", "density", "pH", etc.
#    - Se recomienda aplicar StandardScaler para normalizar estas características.

# features_num_clf_2 → Variables numéricas adicionales
#    - Contiene "fixed acidity" y "residual sugar".
#    - Estas características pueden ser analizadas por separado o combinadas con las anteriores.

features_cat_clf = ["class"]
features_num_clf_1 = ["volatile acidity", "citric acid", "chlorides", "free sulfur dioxide",
                      "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]
features_num_clf_2 = ["fixed acidity", "residual sugar"]


In [300]:
# Definición de características (features) para el problema de regresión:

# features_cat_reg → Variables categóricas
#    - Contiene "class" (tipo de vino: tinto o blanco) y "quality" (calidad del vino).
#    - Estas variables deben ser transformadas con OneHotEncoder.

# features_num_reg → Variables numéricas
#    - Se seleccionan todas las columnas del DataFrame excepto las categóricas.
#    - Esto se logra con una lista por comprensión, excluyendo las columnas en features_cat_reg.
#    - Estas variables deben ser escaladas con StandardScaler.

features_cat_reg = ["class", "quality"]
features_num_reg = [col for col in df.columns if col not in features_cat_reg]

# Ahora features_num_reg contiene solo las variables numéricas.

In [301]:
# Estudio de correlaciones entre las variables numéricas y la variable objetivo de regresión.

# Calculamos la matriz de correlación solo para las variables numéricas.
#    - Se utiliza df[features_num_reg] para excluir las variables categóricas.
#    - numeric_only="True" se usa para evitar advertencias en versiones recientes de pandas.

# Extraemos la correlación absoluta de cada variable con el target de regresión ("alcohol").
#    - np.abs() se usa para obtener la magnitud de la correlación sin importar el signo.
#    - .sort_values(ascending=False) ordena las variables de mayor a menor correlación.

corr = df[features_num_reg].corr(numeric_only="True")
serie_corr = np.abs(corr[target_reg]).sort_values(ascending=False)
serie_corr

# Ahora, serie_corr contiene las variables ordenadas según su grado de correlación con "alcohol".

alcohol                 1.000000
density                 0.682345
residual sugar          0.357459
total sulfur dioxide    0.272970
chlorides               0.260508
free sulfur dioxide     0.188460
pH                      0.116497
fixed acidity           0.091964
volatile acidity        0.036041
citric acid             0.005690
sulphates               0.000412
Name: alcohol, dtype: float64

In [302]:
# Selección de características numéricas basadas en la correlación con el target de regresión.

# Definimos un umbral mínimo de correlación.
#    - r_min = 0.05 significa que solo seleccionaremos variables con correlación mayor a 0.05.

r_min = 0.05

# Seleccionamos las variables numéricas con correlación significativa con el target.
#    - Tomamos solo las variables con correlación absoluta mayor que r_min.
#    - Convertimos los nombres de las columnas en una lista.
#    - Eliminamos el target ("alcohol") de la lista, ya que no es una feature.

features_num_reg_1 = serie_corr[serie_corr > r_min].index.to_list()
features_num_reg_1.remove(target_reg)

# Identificamos las variables numéricas con correlación baja o nula con el target.
#    - Excluimos las variables en features_num_reg_1.
#    - Excluimos el target y las variables categóricas.

features_num_reg_2 = [col for col in df.columns if col not in features_num_reg_1 and col != target_reg
                       and col not in features_cat_reg]

# Ahora:
# - features_num_reg_1 contiene las variables más correlacionadas con "alcohol".
# - features_num_reg_2 contiene las variables menos correlacionadas.

In [303]:
features_num_reg_1

['density',
 'residual sugar',
 'total sulfur dioxide',
 'chlorides',
 'free sulfur dioxide',
 'pH',
 'fixed acidity']

In [304]:
# Definición del problema de clasificación.

# target_clf → Variable objetivo para clasificación.
#    - Se ha definido previamente como "quality".
#    - Representa la calidad del vino en una escala categórica.
#    - Es un problema de clasificación **multiclase** (7 categorías posibles).

target_clf

'quality'

In [305]:
# Definición de las características categóricas para el problema de clasificación.

# features_cat_clf → Variables categóricas a incluir en el modelo de clasificación.
#    - Contiene la variable "class" (tipo de vino: tinto o blanco).
#    - Se debe transformar con OneHotEncoder para convertirla en variables numéricas.

features_cat_clf

['class']

In [306]:
# Definición de características numéricas principales para clasificación.

# features_num_clf_1 → Variables numéricas seleccionadas para el modelo de clasificación.
#    - Incluye medidas químicas clave del vino:
#      - "volatile acidity", "citric acid", "chlorides", "free sulfur dioxide",
#      - "total sulfur dioxide", "density", "pH", "sulphates", "alcohol".
#    - Se recomienda aplicar StandardScaler para normalizar estas características antes del modelado.

features_num_clf_1

['volatile acidity',
 'citric acid',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [307]:
# Definición del problema de regresión.

# target_reg → Variable objetivo para regresión.
#    - Se ha definido previamente como "alcohol".
#    - Representa el porcentaje de alcohol en el vino (variable continua).
#    - Es un problema de regresión, ya que el target es numérico y continuo.

target_reg

'alcohol'

In [308]:
# Definición de las características categóricas para el problema de regresión.

# features_cat_reg → Variables categóricas a incluir en el modelo de regresión.
#    - Contiene:
#      - "class" (tipo de vino: tinto o blanco).
#      - "quality" (calidad del vino en una escala categórica).
#    - Ambas variables deben ser transformadas con OneHotEncoder para su uso en modelos numéricos.

features_cat_reg

['class', 'quality']

In [309]:
# Definición de características numéricas principales para regresión.

# features_num_reg_1 → Variables numéricas con mayor correlación con el target de regresión ("alcohol").
#    - Se han seleccionado aquellas con correlación absoluta > r_min (0.05).
#    - Estas variables son las más relevantes para predecir el contenido de alcohol en el vino.
#    - Se recomienda aplicar StandardScaler para normalizar estas características antes del modelado.

features_num_reg_1

['density',
 'residual sugar',
 'total sulfur dioxide',
 'chlorides',
 'free sulfur dioxide',
 'pH',
 'fixed acidity']

In [310]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Definición de columnas a incluir y excluir en el modelo de clasificación.

# columns_to_keep_clf → Columnas que se mantendrán en el modelo de clasificación.
#    - Incluye:
#      - target_clf (variable objetivo de clasificación: "quality").
#      - features_num_clf_1 (variables numéricas relevantes).
#      - features_cat_clf (variables categóricas a transformar con OneHotEncoder).

# columns_to_exclude_clf → Columnas que se excluirán del modelo de clasificación.
#    - Se obtienen eliminando de df.columns las variables incluidas en columns_to_keep_clf.
#    - Estas columnas no serán utilizadas en el modelo.

columns_to_keep_clf =   features_num_clf_1 + features_cat_clf

columns_to_exclude_clf = [col for col in df.columns if col not in columns_to_keep_clf] + [target_clf] 

columns_to_exclude_clf


['fixed acidity', 'residual sugar', 'quality', 'quality']

In [311]:
# Definición de columnas a incluir y excluir en el modelo de regresión.

# columns_to_keep_reg → Columnas que se mantendrán en el modelo de regresión.
#    - Incluye:
#      - target_reg (variable objetivo de regresión: "alcohol").
#      - features_num_reg_1 (variables numéricas más correlacionadas con el target).
#      - features_cat_reg (variables categóricas a transformar con OneHotEncoder).

# columns_to_exclude_reg → Columnas que se excluirán del modelo de regresión.
#    - Se obtienen eliminando de df.columns las variables incluidas en columns_to_keep_reg.
#    - Estas columnas no serán utilizadas en el modelo.

columns_to_keep_reg = features_num_reg_1 + features_cat_reg

columns_to_exclude_reg = [col for col in df.columns if col not in columns_to_keep_reg]

columns_to_exclude_reg

['volatile acidity', 'citric acid', 'sulphates', 'alcohol']

In [312]:
from sklearn.preprocessing import FunctionTransformer

# Definición de Pipelines para preprocesamiento de datos en clasificación.

# cat_pipeline → Preprocesamiento de variables categóricas.
#    - "Impute_Mode": Imputa valores faltantes con la moda (valor más frecuente).
#    - "OHEncoder": Aplica OneHotEncoder, ignorando categorías desconocidas.

# logaritmica → Transformación logarítmica de variables numéricas.
#    - Usa FunctionTransformer con np.log1p para estabilizar distribuciones sesgadas.
#    - feature_names_out="one-to-one" mantiene los nombres originales de las características.

# num_pipeline → Preprocesamiento de variables numéricas.
#    - "Impute_Mean": Imputa valores faltantes con la media.
#    - "logaritmo": Aplica la transformación logarítmica definida antes.
#    - "SScaler": Aplica StandardScaler para normalizar las variables numéricas.

# imputer_step_cat → ColumnTransformer para aplicar los Pipelines según el tipo de variable.
#    - "Process_Numeric": Aplica num_pipeline a features_num_clf_1 (variables numéricas).
#    - "Process_Categorical": Aplica cat_pipeline a features_cat_clf (variables categóricas).
#    - "Exclude": Elimina las columnas en columns_to_exclude_clf.
#    - remainder="passthrough": Mantiene cualquier otra columna sin modificar.

# pipe_missings_cat → Pipeline final que aplica el ColumnTransformer imputer_step_cat.

cat_pipeline = Pipeline(
    [("Impute_Mode", SimpleImputer(strategy="most_frequent")),  # Imputación con la moda
     ("OHEncoder", OneHotEncoder(handle_unknown='ignore'))  # Manejar categorías desconocidas
    ]
)

logaritmica = FunctionTransformer(np.log1p, feature_names_out="one-to-one") # Esto le indica al Pipeline que el número de características no cambia y que puede usar los nombres originales.

num_pipeline = Pipeline(
    [("Impute_Mean", SimpleImputer(strategy = "mean")), # prevision que en el futuro lleguen datos faltantes
     ("logaritmo", logaritmica),
     ("SScaler", StandardScaler()),
    ]
)

imputer_step_cat = ColumnTransformer(
    [("Process_Numeric", num_pipeline,features_num_clf_1), # feature_numericas seleccionadas para clasificación
     ("Process_Categorical", cat_pipeline, features_cat_clf), # feature_categoriacas seleccionadas para regresión
     ("Exclude", "drop", columns_to_exclude_clf)
    ], remainder = "passthrough"
    )

pipe_missings_cat = Pipeline([("first_stage", imputer_step_cat)])

In [313]:
from sklearn.preprocessing import FunctionTransformer

# Definición de Pipelines para preprocesamiento de datos en regresión.

# cat_pipeline → Preprocesamiento de variables categóricas.
#    - "Impute_Mode": Imputa valores faltantes con la moda (valor más frecuente).
#    - "OHEncoder": Aplica OneHotEncoder, ignorando categorías desconocidas.

# logaritmica → Transformación logarítmica de variables numéricas.
#    - Usa FunctionTransformer con np.log1p para estabilizar distribuciones sesgadas.
#    - feature_names_out="one-to-one" mantiene los nombres originales de las características.

# num_pipeline → Preprocesamiento de variables numéricas.
#    - "Impute_Mean": Imputa valores faltantes con la media.
#    - "logaritmo": Aplica la transformación logarítmica definida antes.
#    - "SScaler": Aplica StandardScaler para normalizar las variables numéricas.

# imputer_step_reg → ColumnTransformer para aplicar los Pipelines según el tipo de variable.
#    - "Process_Numeric": Aplica num_pipeline a features_num_reg_1 (variables numéricas seleccionadas para regresión).
#    - "Process_Categorical": Aplica cat_pipeline a features_cat_reg (variables categóricas seleccionadas para regresión).
#    - "Exclude": Elimina las columnas en columns_to_exclude_reg.
#    - remainder="passthrough": Mantiene cualquier otra columna sin modificar.

# pipe_missings_reg → Pipeline final que aplica el ColumnTransformer imputer_step_reg.

cat_pipeline = Pipeline(
    [("Impute_Mode", SimpleImputer(strategy="most_frequent")),  # Imputación con la moda
     ("OHEncoder", OneHotEncoder(handle_unknown='ignore'))  # Manejar categorías desconocidas
    ]
)

logaritmica = FunctionTransformer(np.log1p, feature_names_out="one-to-one") # Esto le indica al Pipeline que el número de características no cambia y que puede usar los nombres originales.

num_pipeline = Pipeline(
    [("Impute_Mean", SimpleImputer(strategy = "mean")), # prevision que en el futuro lleguen datos faltantes
     ("logaritmo", logaritmica),
     ("SScaler", StandardScaler()),
    ]
)

imputer_step_reg = ColumnTransformer(
    [("Process_Numeric", num_pipeline,features_num_reg_1), # feature_numericas seleccionadas para clasificación
     ("Process_Categorical", cat_pipeline, features_cat_reg), # feature_categoriacas seleccionadas para regresión
     ("Exclude", "drop", columns_to_exclude_reg)
    ], remainder = "passthrough"
    )

pipe_missings_reg = Pipeline([("first_stage", imputer_step_reg)])

In [314]:
# Aplicación del pipeline de preprocesamiento a los datos de prueba.

# pipe_missings_cat.fit_transform(df_test)
#    - Aplica el pipeline de preprocesamiento definido para clasificación.
#    - Imputa valores faltantes en variables numéricas y categóricas.
#    - Transforma variables categóricas con OneHotEncoder.
#    - Aplica escalado y transformación logarítmica a las variables numéricas.

# pipe_df_test → Contiene los datos de prueba preprocesados y listos para el modelo.

pipe_df_test = pipe_missings_cat.fit_transform(df_test)

pipe_df_test

array([[-0.06561088, -0.27783928, -0.1913355 , ..., -1.03455179,
         0.        ,  1.        ],
       [ 0.061259  , -0.49980343, -0.57998343, ...,  1.48951019,
         0.        ,  1.        ],
       [-0.06561088,  0.2893635 , -0.68457525, ...,  1.63689007,
         0.        ,  1.        ],
       ...,
       [ 0.061259  ,  1.14300849, -0.11427313, ...,  1.33989719,
         1.        ,  0.        ],
       [ 0.78484693, -1.03933617,  0.41908799, ...,  0.79764424,
         1.        ,  0.        ],
       [ 0.18624926, -1.28052007, -0.52783916, ..., -0.93914749,
         0.        ,  1.        ]])

In [315]:
# Aplicación del pipeline de preprocesamiento a los datos de entrenamiento.

# pipe_missings_cat.fit_transform(df)
#    - Aplica el pipeline de preprocesamiento definido para clasificación.
#    - Imputa valores faltantes en variables numéricas y categóricas.
#    - Transforma variables categóricas con OneHotEncoder.
#    - Aplica escalado y transformación logarítmica a las variables numéricas.

# pipe_df → Contiene los datos de entrenamiento preprocesados y listos para el modelo.

pipe_df = pipe_missings_cat.fit_transform(df)
pipe_df

array([[-1.05515355,  0.59288805, -0.20549433, ...,  0.72017181,
         1.        ,  0.        ],
       [-1.05515355,  2.5608768 , -0.37024226, ..., -1.3127524 ,
         0.        ,  1.        ],
       [ 1.11199789, -0.50565223, -0.5026097 , ...,  0.31263542,
         0.        ,  1.        ],
       ...,
       [-0.90744764,  0.46264335, -0.10702207, ...,  0.47768498,
         0.        ,  1.        ],
       [ 0.5098244 , -0.14782501, -0.7688788 , ...,  1.99147821,
         0.        ,  1.        ],
       [-1.05515355, -0.21826777, -0.53578111, ...,  0.72017181,
         0.        ,  1.        ]])

In [316]:
# Conversión de los datos preprocesados a un DataFrame con nombres de columnas.

# pd.DataFrame(pipe_df, columns=pipe_missings_cat.get_feature_names_out())
#    - Convierte los datos preprocesados (pipe_df) en un DataFrame de pandas.
#    - Asigna los nombres de las características generados por el ColumnTransformer.
#    - Permite verificar que las transformaciones se aplicaron correctamente.

# df_check → Contiene los datos de entrenamiento preprocesados en formato DataFrame, con nombres de columnas.

df_check = pd.DataFrame(pipe_df, columns= pipe_missings_cat.get_feature_names_out())
df_check

,Process_Numeric__volatile acidity,Process_Numeric__citric acid,Process_Numeric__chlorides,Process_Numeric__free sulfur dioxide,Process_Numeric__total sulfur dioxide,Process_Numeric__density,Process_Numeric__pH,Process_Numeric__sulphates,Process_Numeric__alcohol,Process_Categorical__class_red,Process_Categorical__class_white
0,-1.055154,0.592888,-0.205494,0.603446,-0.505241,0.427700,0.706744,2.584726,0.720172,1.0,0.0
1,-1.055154,2.560877,-0.370242,0.820606,0.855001,1.621578,1.011233,0.998990,-1.312752,0.0,1.0
2,1.111998,-0.505652,-0.502610,-1.317236,0.112504,-0.901360,-1.066182,-1.830435,0.312635,0.0,1.0
3,-0.548692,-0.218268,-0.074260,0.564028,0.448623,0.059153,-1.000780,-0.177558,-0.927596,0.0,1.0
4,0.320822,0.657314,1.525362,-2.003136,-3.095378,0.440977,-0.547423,0.391872,1.187890,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5192,-0.619281,1.217401,-0.271299,1.282255,1.111562,0.626829,-0.228277,0.322290,-1.022479,0.0,1.0
5193,-0.762183,-0.218268,0.089083,-0.011858,0.371746,1.276901,1.431729,-0.549087,-0.927596,0.0,1.0
5194,-0.907448,0.462643,-0.107022,0.255061,0.581225,-0.003952,0.645425,0.252264,0.477685,0.0,1.0
5195,0.509824,-0.147825,-0.768879,0.045414,-0.365924,-1.936549,-1.662178,-1.332210,1.991478,0.0,1.0


In [317]:
# Conversión de los datos de prueba preprocesados a un DataFrame con nombres de columnas.

# pd.DataFrame(pipe_df_test, columns=pipe_missings_cat.get_feature_names_out())
#    - Convierte los datos de prueba preprocesados (pipe_df_test) en un DataFrame de pandas.
#    - Asigna los nombres de las características generados por el ColumnTransformer.
#    - Permite verificar que las transformaciones se aplicaron correctamente.

# df_check_test → Contiene los datos de prueba preprocesados en formato DataFrame, con nombres de columnas.

df_check_test = pd.DataFrame(pipe_df_test, columns= pipe_missings_cat.get_feature_names_out())
df_check_test

,Process_Numeric__volatile acidity,Process_Numeric__citric acid,Process_Numeric__chlorides,Process_Numeric__free sulfur dioxide,Process_Numeric__total sulfur dioxide,Process_Numeric__density,Process_Numeric__pH,Process_Numeric__sulphates,Process_Numeric__alcohol,Process_Categorical__class_red,Process_Categorical__class_white
0,-0.065611,-0.277839,-0.191336,0.609936,0.831639,-0.433274,-1.146519,-0.036117,-1.034552,0.0,1.0
1,0.061259,-0.499803,-0.579983,0.200694,0.019098,-1.467980,-0.701377,-0.919534,1.489510,0.0,1.0
2,-0.065611,0.289363,-0.684575,0.441142,0.048148,-1.164893,0.887973,0.514885,1.636890,0.0,1.0
3,-0.661389,0.080702,-0.789574,0.798095,0.594218,-1.437325,0.469593,0.104306,1.636890,0.0,1.0
4,0.061259,-0.351239,-0.475795,1.255666,0.502182,-1.812097,-1.339665,-0.395421,1.414987,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1295,-0.292330,0.080702,0.166421,0.349361,1.050532,1.146368,-1.339665,-0.542576,-1.130887,0.0,1.0
1296,0.842435,0.888607,2.218732,0.527821,-0.940203,1.268510,1.006277,2.132366,-0.475482,1.0,0.0
1297,0.061259,1.143008,-0.114273,-2.309748,-3.358761,-0.015075,-0.078471,1.102750,1.339897,1.0,0.0
1298,0.784847,-1.039336,0.419088,-1.861014,-3.091298,-0.127250,2.329225,0.104306,0.797644,1.0,0.0


In [ ]:
### EVALUACIÓN DE MODELOS DE CLASIFICACIÓN
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier


In [ ]:
logistic_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),
     ("Modelo", LogisticRegression(max_iter=10000))
    ])

random_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),
     ("Modelo", RandomForestClassifier())
    ])

Kneighbors_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),
     ("Modelo", KNeighborsClassifier())
    ])

LGBM_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),
     ("Modelo", LGBMClassifier(verbose=-1))
    ])


for name, pipe in zip(["logistic","randomF", "Kneighbors","LGBM"],[logistic_pipeline, random_pipeline, Kneighbors_pipeline, LGBM_pipeline]):
    resultado_
    resultado = cross_val_score(pipe, df, y_train_cat, cv = 5, scoring = "accuracy")
    print(f"{name}: {np.mean(resultado):.4f}")
    print(resultado)

logistic: 0.5470
[0.52980769 0.55769231 0.54186718 0.54764196 0.55822907]
randomF: 0.6611
[0.66442308 0.66153846 0.6612127  0.66506256 0.65351299]
Kneighbors: 0.5449
[0.55288462 0.55480769 0.53224254 0.54475457 0.53994225]
LGBM: 0.6423
[0.65       0.63942308 0.64773821 0.63522618 0.63907603]


In [323]:
### EVALUACION DE MODELOS DE REGRESION

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [ ]:
randomreg_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),
     ("Modelo", RandomForestRegressor())
    ])

XGB_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),
     ("Modelo", XGBRegressor())
    ])

LGBMreg_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),
     ("Modelo", LGBMRegressor(verbose=-1))
    ])


for name, pipe in zip(["RandomF","XGB","LGBM"],[randomreg_pipeline, XGB_pipeline, LGBMreg_pipeline]):
    resultado_reg = cross_val_score(pipe, df, y_train_reg, cv = 5, scoring = "neg_mean_absolute_percentage_error")
    print(f"{name}: {-np.mean(resultado_reg):.4f}")
    print(resultado_reg)

RandomF: 0.0276
[-0.02877444 -0.0284786  -0.02691515 -0.02693453 -0.02710483]
XGB: 0.0271
[-0.02807295 -0.02796904 -0.02587827 -0.02658056 -0.02686444]
LGBM: 0.0287
[-0.02986407 -0.02965162 -0.02812008 -0.02791631 -0.02802684]


In [ ]:
### EVALUACION DE PARAMETROS DE CLASIFICACION

In [337]:
from sklearn.model_selection import GridSearchCV

# Definimos sus hiperparametros
reg_log_param = {    
                 "penalty": [None,"l2"], 
                 "C": np.logspace(0, 4, 10)
                }

rand_forest_param = {
    'n_estimators': [10, 100, 200, 400],
    'max_depth': [1,2,4,8],
    'max_features': [1, 2, 3]
    }

param_grid_lgbm = {
    'Modelo__num_leaves': [15, 31, 50],
    'Modelo__learning_rate': [0.01, 0.05, 0.1],
    'Modelo__n_estimators': [50, 100, 200],
    'Modelo__max_depth': [5, 10, 15],
}

param_grid_knn = {
    "Modelo__n_neighbors": [3, 5, 7, 9, 11],
    "Modelo__weights": ['uniform', 'distance'],
    "Modelo__metric": ["euclidean", "manhattan"]
}


cv = 5

gs_reg_log = GridSearchCV(LogisticRegression(),
                            reg_log_param,
                            cv=cv,
                            scoring="accuracy",
                            verbose=1,
                            n_jobs=-1)

gs_rand_forest = GridSearchCV(RandomForestClassifier(),
                            rand_forest_param,
                            cv=cv,
                            scoring="accuracy",
                            verbose=1,
                            n_jobs=-1)

gs_lgb = GridSearchCV(LGBMClassifier(),
                        param_grid_lgbm,
                        cv=cv,
                        scoring="accuracy",
                        verbose=1,
                        n_jobs=-1)

grid_search_knn = GridSearchCV(estimator=Kneighbors_pipeline,
                               param_grid=param_grid_knn,
                               scoring='accuracy',
                               cv=5,
                               verbose=1,
                               n_jobs=-1)


pipe_grids_cat = {"gs_reg_log":gs_reg_log,
         "gs_rand_forest":gs_rand_forest,
         "gs_lgb":gs_lgb,
         "gs_Knn":grid_search_knn}
    


In [338]:
for nombre, grid_search in pipe_grids_cat.items():
    grid_search.fit(df, y_train_cat)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rodri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1223, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\rodri\App